In [ ]:
import optuna
from optuna.visualization import (
    plot_optimization_history,
    plot_param_importances,
    plot_slice,
    plot_parallel_coordinate,
    plot_contour
)
import plotly.io as pio
pio.renderers.default = 'notebook'  # or 'browser'

In [ ]:
study_name = "lstm_regression_v1"
storage = "sqlite:///optuna_studies.db"

study = optuna.load_study(
    study_name=study_name,
    storage=storage
)

print(f"Study: {study.study_name}")
print(f"Direction: {study.direction}")
print(f"Best value: {study.best_value:.6f}")
print(f"Best params: {study.best_params}")
print(f"Trials: {len(study.trials)}")

In [ ]:
fig = plot_optimization_history(study)
fig.update_layout(
    title="Optimization History - LSTM Regression",
    height=500,
    showlegend=True
)
fig.show()

In [ ]:
fig = plot_param_importances(
    study,
    evaluator=optuna.importance.FanovaImportanceEvaluator()
)
fig.update_layout(
    title="Hyperparameter Importance (fANOVA)",
    height=500,
    xaxis_title="Importance Score"
)
fig.show()

In [ ]:
if hasattr(study.trials[0], 'duration'):
    fig = plot_param_importances(
        study,
        target=lambda t: t.duration.total_seconds(),
        target_name="duration"
    )
    fig.update_layout(
        title="Hyperparameter Impact on Computation Time",
        height=500
    )
    fig.show()

In [ ]:
fig = plot_slice(study)
fig.update_layout(
    title="Hyperparameter Slice Plots",
    height=600
)
fig.show()

In [ ]:
key_params = ['look_back', 'n_components', 'learning_rate', 'dropout']
fig = plot_slice(study, params=key_params)
fig.update_layout(
    title="Key Hyperparameter Effects",
    height=400
)
fig.show()

In [ ]:
fig = plot_parallel_coordinate(
    study,
    params=['look_back', 'n_components', 'n_layers', 'n_units', 'dropout', 'learning_rate']
)
fig.update_layout(
    title="Parallel Coordinates - All Hyperparameters",
    height=600
)
fig.show()

In [ ]:
fig = plot_contour(
    study,
    params=['look_back', 'n_components']
)
fig.update_layout(title="Look Back vs N Components")
fig.show()

In [ ]:
fig = plot_contour(
    study,
    params=['learning_rate', 'dropout']
)
fig.update_layout(title="Learning Rate vs Dropout")
fig.show()

In [ ]:
best_trial = study.best_trial

print("="*70)
print("BEST TRIAL ANALYSIS")
print("="*70)
print(f"Trial number: {best_trial.number}")
print(f"Value (RMSE): {best_trial.value:.6f}")
print(f"\nHyperparameters:")
for key, value in best_trial.params.items():
    print(f"  {key}: {value}")

print(f"\nUser Attributes:")
for key, value in best_trial.user_attrs.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.6f}")
    else:
        print(f"  {key}: {value}")

In [ ]:
pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]

print(f"Complete trials: {len(complete_trials)}")
print(f"Pruned trials: {len(pruned_trials)}")
print(f"Pruning rate: {len(pruned_trials)/len(study.trials)*100:.1f}%")

# Visualize when pruning happened
import plotly.graph_objects as go

pruned_steps = [len(t.intermediate_values) for t in pruned_trials]

fig = go.Figure()
fig.add_trace(go.Histogram(
    x=pruned_steps,
    name='Pruned Trials',
    nbinsx=20
))
fig.update_layout(
    title="Distribution of Pruning Steps",
    xaxis_title="Step When Pruned",
    yaxis_title="Count",
    height=400
)
fig.show()

In [ ]:
import pandas as pd
import numpy as np

# Extract fold RMSEs from user attributes
fold_data = []
for trial in complete_trials:
    fold_rmses = [
        trial.user_attrs.get(f'fold_{i}_rmse', np.nan)
        for i in range(10)  # Adjust based on your max folds
    ]
    # Remove nans
    fold_rmses = [x for x in fold_rmses if not np.isnan(x)]
    if fold_rmses:
        fold_data.append({
            'trial': trial.number,
            'mean_rmse': trial.value,
            'std_rmse': np.std(fold_rmses),
            'cv_stability': np.std(fold_rmses) / np.mean(fold_rmses)
        })

stability_df = pd.DataFrame(fold_data)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=stability_df['mean_rmse'],
    y=stability_df['cv_stability'],
    mode='markers',
    marker=dict(
        size=10,
        color=stability_df['trial'],
        colorscale='Viridis',
        showscale=True,
        colorbar=dict(title="Trial #")
    ),
    text=[f"Trial {t}" for t in stability_df['trial']],
    hovertemplate='<b>%{text}</b><br>Mean RMSE: %{x:.4f}<br>CV Stability: %{y:.4f}<extra></extra>'
))

fig.update_layout(
    title="Cross-Validation Stability vs Performance",
    xaxis_title="Mean RMSE",
    yaxis_title="CV Stability (Std/Mean)",
    height=500
)
fig.show()

In [ ]:
import yaml

best_config = {
    'best_trial_number': best_trial.number,
    'best_value': float(best_trial.value),
    'best_params': best_trial.params,
    'timestamp': str(best_trial.datetime_complete)
}

with open('../configs/best_lstm_regression_params.yaml', 'w') as f:
    yaml.dump(best_config, f, default_flow_style=False)

print("✓ Saved best parameters to configs/best_lstm_regression_params.yaml")